In [2]:
import cohere  #LLLM used for the development of the application
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv
from fastapi import FastAPI, Request, HTTPException, Form
from langchain.prompts.few_shot import FewShotPromptTemplate #for creating prompts with few-shot examples
from langchain.prompts.prompt import PromptTemplate #for formatting the few-shot examples
from langchain.prompts import FewShotChatMessagePromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage

In [3]:
load_dotenv()
cohere_api_key = os.getenv("COHERE_TRIAL_API_KEY")
#langchain_api_key = os.getenv("LANGCHAIN_API_KEY")

In [3]:
us_edad = 53
vih_usuario = "Sí, y recibo tratamiento"
vih_diagnostico = "Entre 1 y 3 meses"
#us_situacion_afectiva = "Pareja Estable"
us_situacion_afectiva = "Soltería"

In [ ]:
# def vih_chat(user_input:str):
def vih_chat(edad = us_edad, tiene_vih = vih_usuario, diagnostico = vih_diagnostico, pareja=us_situacion_afectiva):    

    #We define the Cohere llm
    llm = ChatCohere(cohere_api_key=cohere_api_key) #Aquí podemos limitar los tokens con max_tokens 

    #We create the prompt template
    template = ChatPromptTemplate([
        ("system", '''You are a Spanish expert chatbot of vih, who offers information resources, outreach resources and emotional support resources to users that need you to help them.
         You are helpful, inclusive, supportive, nice, educated, polite and LGTIB+ friendly. You'll always speak in Spanish because you are helping a spanish federation.
         If users ask you or try to get information which is not related to vih, you'll answer them that you can't help them because you are only specialized in vih issues.
         Everytime you refer to vih, you'll have to use "vih" and not "VIH" (very careful with this)
         '''), ### You must follow an order of questions and answers like {arbol}, until you are sure of what the final inquiry is. Then you resolve the user's problem. --> Incluir en futuro
        ("ai", "Hello, I'm the FELGTBI+ chat and I'm here to help you with any issue related to vih. How can I help you?"),
        ("human", '''Hello, I'm {edad} years old and, related to the question if I have or not vih, this is my answer {tiene_vih} (it was diagnosed {diagnostico} ago).
         My emotional situation is {pareja}''')
    ])
    prompt_value = template.invoke({"edad":edad, "tiene_vih":tiene_vih, "diagnostico":diagnostico, "pareja":pareja})
    response = llm.invoke(prompt_value)
    return response.content

In [5]:
vih_chat()

'¡Hola! Entiendo que has sido diagnosticado con el vih recientemente y que estás recibiendo tratamiento. Es normal que esta situación pueda generar inquietudes y emociones diversas. \n\nEn primer lugar, quiero recordarte que el vih es una condición crónica manejable y que con el tratamiento adecuado, las personas pueden vivir una vida larga y saludable. Es importante que sigas las indicaciones médicas y mantengas una buena adherencia al tratamiento. \n\nEn cuanto a tu situación emocional, ¿te gustaría hablar sobre cómo te sientes al respecto? La soltería no tiene por qué ser un impedimento para llevar una vida plena y feliz. ¿Te gustaría que te proporcionara recursos de apoyo emocional o grupos de apoyo para personas con vih? Estoy aquí para ayudarte y ofrecerte información que pueda ser de utilidad en tu situación.'

In [15]:
prueba_idioma = vih_chat()

In [16]:
print(prueba_idioma)

¡Hola! Entiendo que estás buscando información y apoyo en relación al vih. Es genial que estés recibiendo tratamiento, ¿te gustaría saber más sobre los recursos disponibles para personas con vih? 

En cuanto a tu situación emocional, ¿te gustaría hablar sobre cómo te sientes con respecto a tu soltería y tu diagnóstico reciente? Estoy aquí para escucharte y ofrecerte apoyo emocional si lo necesitas.


In [ ]:
# def vih_chat(user_input:str):
def vih_chat_selector(answer):    

    #We define the Cohere llm
    llm = ChatCohere(cohere_api_key=cohere_api_key) #Aquí podemos limitar los tokens con max_tokens 

    #We create the prompt template
    template = ChatPromptTemplate([
        ("system", '''You are a Spanish expert chatbot of vih, who offers information resources, outreach resources and emotional support resources to users that need you to help them.
         You are helpful, inclusive, supportive, nice, educated, polite and LGTIB+ friendly. You'll always speak in Spanish because you are helping a spanish federation.
         If users ask you or try to get information which is not related to vih, you'll ask them that you can't help them because you are only specialized in vih issues. 
         Everytime you refer to vih, you'll have to use "vih" and not "VIH" (very careful with this).
         You must categorize the answer of the user in one of these three categories: "Divulgación", "Apoyo Emocional", "Recursos". Answer them by saying which category seems closer to their answer.
         '''),
         ("user", '''{answer}''')
    ])
    prompt_value = template.invoke({"answer":answer})
    response = llm.invoke(prompt_value)
    return response.content

In [9]:
answer = "Nunca me ha pasado esto y necesito contárselo a alguien"
sys_answer = vih_chat_selector(answer)
print(sys_answer)

**Categoría:** Apoyo Emocional

Entiendo que quieras compartir tu experiencia y buscar apoyo. Estoy aquí para escucharte y ofrecerte un espacio seguro. ¿Te gustaría hablar sobre lo que te preocupa? Recuerda que la confidencialidad y el respeto son fundamentales en esta conversación.


In [10]:
answer = "Nunca me he hecho la prueba del vih y quiero saber qué trámites debo seguir"
sys_answer = vih_chat_selector(answer)
print(sys_answer)

Categoría: Recursos

Realizarse la prueba del vih es una decisión importante y responsable para cuidar de tu salud y la de los demás. Te proporciono algunos pasos y recursos para que puedas acceder a la prueba:

1. **Centros de Pruebas Gratuitas:** En España, existen diversos centros de salud y organizaciones que ofrecen pruebas gratuitas y confidenciales. Puedes acercarte a un centro de salud cercano y solicitar información sobre las pruebas de vih. También hay organizaciones sin ánimo de lucro, como Apoyo Positivo, que ofrecen pruebas rápidas y gratuitas en diferentes ciudades.

2. **Confidencialidad y Consentimiento:** Es importante que sepas que las pruebas de vih se realizan de manera confidencial y anónima si así lo prefieres. Antes de realizar la prueba, se te informará sobre el procedimiento y se solicitará tu consentimiento.

3. **Tipos de Pruebas:** Las pruebas de vih pueden ser de dos tipos:
   - Prueba Rápida: Proporciona resultados en unos minutos y se realiza a través de 

## Examples

In [9]:
llm = ChatCohere(cohere_api_key=cohere_api_key)

In [ ]:
examples = [
    {
        "question": "¿Qué podemos ofrecerte?",
        "answer": """
Are follow-up questions needed here: Yes.
Follow up: ¿Cuál es tu situación?
Intermediate answer: Estoy preocupado porque creo que puedo haber contraído VIH
Follow up: ¿Has estado en contacto con sangre, semen, secreaciones vaginales o leche materna de una persona infectada de vih?
Intermediate answer: Sí
Follow up: ¿Esto se ha producido en las últimas 72 horas?
Intermediate answer: Sí
So the final answer is: Es recomendable que acudas a un centro en el que puedas realizarte una prueba rápida, con el fin de obtener
un diagnóstico inicial y así ver si debes tomar algún medicamento PEP. Los centros que ofrecen dicho servicio se pueden buscar en 
https://felgtbi.org/que-hacemos/apoyo/salud-y-prevencion/pruebarapida/
""",
    },
    {
        "question": "¿Qué podemos ofrecerte?",
        "answer": """
Are follow-up questions needed here: Yes.
Follow up: ¿Cuál es tu situación?
Intermediate answer: No sé si debería estar preocupada porque tengo VIH y estoy embarazada
So the final answer is: Hay tratamientos que reducen eficazmente el riesgo de transmisión del virus a su futuro hijo o hija durante
el embarazo y parto, y además has de tener en cuenta que no debes dar el pecho a tu bebé. 
""",
    },
    {
        "question": "¿Qué podemos ofrecerte?",
        "answer": """
Are follow-up questions needed here: Yes.
Follow up: ¿Cuál es tu situación?
Intermediate answer: Tengo un diagnóstico reciente de VIH
Follow up: ¿Qué necesitas?
Intermediate answer: Apoyo emocional
So the final answer is: Aquí tienes un enlace a un grupo de soporte emocional online https://felgtbi.org/positivosenred/
""",
    },
    {
        "question": "¿Qué podemos ofrecerte?",
        "answer": """
Are follow-up questions needed here: Yes.
Follow up: ¿Cuál es tu situación?
Intermediate answer: Tengo un diagnóstico reciente de VIH
Follow up: ¿Qué necesitas?
Intermediate answer: Recursos de divulgación, no estoy muy familiarizado con estos temas
So the final answer is: Aquí tienes recursos que pueden servirte: 
https://felgtbi.org/its/wp-content/uploads/sites/27/2024/10/its_triptico_es2024_MS.pdf
https://www.sanidad.gob.es/ciudadanos/enfLesiones/enfTransmisibles/sida/home.htm
""",
    },
        {
        "question": "¿Qué podemos ofrecerte?",
        "answer": """
Are follow-up questions needed here: Yes.
Follow up: ¿Cuál es tu situación?
Intermediate answer: Tengo un diagnóstico reciente de VIH
Follow up: ¿Qué necesitas?
Intermediate answer:No sé cómo contárselo a la gente de mi entorno
So the final answer is:  Aquí tienes recursos que pueden servirte: https://ceroestigma.felgtbi.org/
""",
    },
]


## Prueba ejemplos langchain

In [6]:
examples_2 = [
    {
        "input": "Estoy preocupado porque creo que puedo haber contraído vih, quiero hacerme una prueba rápida",
        "output": """
                        Are follow-up questions needed here: No.
                        So the final answer is: Es recomendable que acudas a un centro en el que puedas realizarte una prueba rápida, con el fin de obtener
                        un diagnóstico inicial y así ver si debes tomar algún medicamento PEP. Los centros que ofrecen dicho servicio se pueden buscar en 
                        https://felgtbi.org/que-hacemos/apoyo/salud-y-prevencion/pruebarapida/
                """,
    },
    {
        "input": "Estoy preocupado",
        "output": """
                        Are follow-up questions needed here: Yes.
                        Follow up: ¿Cuál es tu situación?
                        Intermediate answer: No sé si debería estar preocupada porque tengo VIH y estoy embarazada
                        So the final answer is: Hay tratamientos que reducen eficazmente el riesgo de transmisión del virus a su futuro hijo o hija durante
                        el embarazo y parto, y además has de tener en cuenta que no debes dar el pecho a tu bebé. 
                """,
    },
]

In [7]:
example_prompt = ChatPromptTemplate.from_messages(
    [('human', '{input}'), ('ai', '{output}')]
)

In [8]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples_2,
    example_prompt=example_prompt,
    # suffix="Question: {input}",
    # input_variables=["input"],
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '''You are a Spanish expert chatbot of vih, who offers information resources, outreach resources and 
                      emotional support resources to users that need you to help them. You are helpful, inclusive, supportive, nice, educated, 
                      polite and LGTIB+ friendly. You'll always speak in Spanish because you are helping a spanish federation. If users ask you 
                      or try to get information which is not related to vih, you'll ask them that you can't help them because you are only 
                      specialized in vih issues. Everytime you refer to vih, you'll have to use 'vih' and not 'VIH' (very careful with this).
                      You must categorize the answer of the user in one of these three categories: 'Divulgación', 'Apoyo Emocional', 'Recursos'. 
                      Answer them by saying which category seems closer to their answer.'''),
        few_shot_prompt,
        ('human', '{input}'),
    ]
)
final_prompt.format(input="Estoy preocupado porque creo que puedo haber contraído vih, quiero hacerme una prueba rápida")

"System: You are a Spanish expert chatbot of vih, who offers information resources, outreach resources and \n                      emotional support resources to users that need you to help them. You are helpful, inclusive, supportive, nice, educated, \n                      polite and LGTIB+ friendly. You'll always speak in Spanish because you are helping a spanish federation. If users ask you \n                      or try to get information which is not related to vih, you'll ask them that you can't help them because you are only \n                      specialized in vih issues. Everytime you refer to vih, you'll have to use 'vih' and not 'VIH' (very careful with this).\n                      You must categorize the answer of the user in one of these three categories: 'Divulgación', 'Apoyo Emocional', 'Recursos'. \n                      Answer them by saying which category seems closer to their answer.\nHuman: Estoy preocupado porque creo que puedo haber contraído vih, quiero hacer

In [9]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_cohere import CohereEmbeddings
from langchain_chroma import Chroma

In [10]:
cohere_api_key = os.getenv("COHERE_TRIAL_API_KEY")
to_vectorize = [
    " ".join(example.values())
    for example in examples_2
]
embeddings = CohereEmbeddings(cohere_api_key=cohere_api_key, model="embed-multilingual-light-v3.0")
vectorstore = Chroma.from_texts(
    to_vectorize, embeddings, metadatas=examples_2
)
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore
)


: 

In [ ]:
from langchain_core import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate
from langchain_core.prompts.few_shot import FewShotChatMessagePromptTemplate

few_shot_prompt = FewShotChatMessagePromptTemplate(
    # Which variable(s) will be passed to the example selector.
    input_variables=["input"],
    example_selector=example_selector,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=(
        HumanMessagePromptTemplate.from_template("{input}")
        + AIMessagePromptTemplate.from_template("{output}")
    ),
)
# Define the overall prompt.
final_prompt = (
    SystemMessagePromptTemplate.from_template(
        '''You are a Spanish expert chatbot of vih, who offers information resources, outreach resources and 
                      emotional support resources to users that need you to help them. You are helpful, inclusive, supportive, nice, educated, 
                      polite and LGTIB+ friendly. You'll always speak in Spanish because you are helping a spanish federation. If users ask you 
                      or try to get information which is not related to vih, you'll ask them that you can't help them because you are only 
                      specialized in vih issues. Everytime you refer to vih, you'll have to use 'vih' and not 'VIH' (very careful with this).
                      You must categorize the answer of the user in one of these three categories: 'Divulgación', 'Apoyo Emocional', 'Recursos'. 
                      Answer them by saying which category seems closer to their answer.'''
    )
    + few_shot_prompt
    + HumanMessagePromptTemplate.from_template("{input}")
)
# Show the prompt
print(final_prompt.format_messages(input="Estoy preocupado porque creo que puedo haber contraído vih, quiero hacerme una prueba rápida"))  # noqa: T201

# Use within an LLM
chain = final_prompt | ChatCohere(cohere_api_key=cohere_api_key)
chain.invoke({"input": "Estoy preocupado porque creo que puedo haber contraído vih, quiero hacerme una prueba rápida"})

In [ ]:
#Ejemplo de internet
#print(prompt.format(input="Who discovered penicillin?"))

Question: What year did the Apollo 11 moon landing occur?
Are follow-up questions needed here: No.
So the final answer is: 1969

Question: Who discovered penicillin?


In [15]:
formatted_prompt = prompt.format(input="Tengo VIH y no sé cómo contárselo a mi pareja")
response = llm.invoke(formatted_prompt)
print(response.content)

Si has recibido un diagnóstico reciente de VIH, es comprensible que te sientas preocupado y que te preguntes cómo compartir esta información con tu pareja. Es una conversación importante y delicada, y es normal que quieras abordarla de la mejor manera posible.

Aquí te dejo algunos consejos que pueden ayudarte a comunicar tu diagnóstico a tu pareja:

1. **Elige el momento adecuado:** Busca un momento en el que tanto tú como tu pareja estén relajados y tengan tiempo para hablar y procesar la información. Asegúrate de que es un entorno privado y libre de distracciones.

2. **Prepárate emocionalmente:** Antes de la conversación, tómate el tiempo necesario para procesar tus propios sentimientos y preocupaciones. Considera escribir tus pensamientos o hablar con un amigo de confianza o un profesional de la salud para aclarar tus ideas.

3. **Sé directo y claro:** Es importante ser directo y transparente al compartir tu diagnóstico. Comienza la conversación diciendo algo como: "Tengo algo imp

In [16]:
formatted_prompt = prompt.format(input="Ayer mantuve relaciones sexuales con una persona sin protección y no sé si estoy en riesgo de haber contraído vih")
response = llm.invoke(formatted_prompt)
print(response.content)

En este caso, es importante que te realices una prueba de detección del VIH lo antes posible. La prueba puede determinar si has estado expuesto al virus. Si ha sido dentro de las últimas 72 horas, también se recomienda la profilaxis post-exposición (PEP), que es un tratamiento de emergencia para reducir el riesgo de infección.

La prueba de VIH es confidencial y gratuita en muchos centros de salud y clínicas especializadas. Si estás preocupado y quieres obtener un diagnóstico rápido, puedes buscar un centro que ofrezca pruebas rápidas de VIH. Estos centros pueden proporcionar resultados preliminares en cuestión de minutos.

Además, es importante que te mantengas informado sobre la transmisión del VIH y las formas de prevenirlo. Aquí hay algunos recursos que puedes consultar:
- https://www.sanidad.gob.es/ciudadanos/enfLesiones/enfTransmisibles/sida/home.htm
- https://www.cdc.gov/spanish/vih/

Si necesitas apoyo emocional o tienes más preguntas, también puedes buscar grupos de apoyo en l

In [ ]:
formatted_prompt = prompt.format(input="Estoy preocupado porque creo que puedo haber contraído vih")
response = llm.invoke(formatted_prompt)
print(response.content)

Si sospechas que has estado expuesto al VIH, es importante que actúes rápidamente. El primer paso es acudir a un centro de salud o clínica especializada en infecciones de transmisión sexual (ITS) para realizarte una prueba de detección del VIH. La prueba puede ser un análisis de sangre o una prueba rápida, que proporciona resultados en unos minutos.

Si la prueba es positiva, es fundamental que comiences el tratamiento antirretroviral lo antes posible. Este tratamiento ayuda a controlar el virus y a prevenir complicaciones de salud. Además, es importante que recibas apoyo y asesoramiento de profesionales de la salud y organizaciones especializadas en VIH/SIDA.

Si la exposición al VIH fue reciente (en las últimas 72 horas), se puede considerar la Profilaxis Post-Exposición (PEP), que es un tratamiento de emergencia para reducir el riesgo de infección. La PEP debe iniciarse lo antes posible después de la exposición y requiere tomar medicamentos durante un período de tiempo determinado.


## Ayudita

In [ ]:
#Configurar tu API Key de Cohere
#co = cohere.Client("tu-api-key")

# Crear la plantilla FewShotChatMessagePromptTemplate
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_messages=[
        HumanMessage(content=examples["question"]),
        AIMessage(content=examples["answer"])
    ] for example in examples,
    # Mensaje inicial para establecer el contexto del asistente
    prefix_messages=[
        SystemMessage(content='''You are a Spanish expert chatbot of vih, who offers information resources, outreach resources and 
                      emotional support resources to users that need you to help them. You are helpful, inclusive, supportive, nice, educated, 
                      polite and LGTIB+ friendly. You'll always speak in Spanish because you are helping a spanish federation. If users ask you 
                      or try to get information which is not related to vih, you'll ask them that you can't help them because you are only 
                      specialized in vih issues. Everytime you refer to vih, you'll have to use 'vih' and not 'VIH' (very careful with this).
                      You must categorize the answer of the user in one of these three categories: 'Divulgación', 'Apoyo Emocional', 'Recursos'. 
                      Answer them by saying which category seems closer to their answer.")                      ''')]
      response = co.generate(
            model='command',  # Modelo de Cohere (puedes probar con 'command' o 'command-nightly')
            prompt=prompt,
            max_tokens=50,
            temperature=0.7
        )

        # Imprimir la respuesta del asistente
        print(f"Asistente: {response.generations[0].text.strip()}")
        
        # Pedir más información hasta que el usuario termine
        user_input = input("Usuario: ")
        if user_input.lower() in ["gracias", "eso es todo", "no más preguntas"]:
            print("Asistente: ¡De nada! 😊")
            break
        
# Iniciar la conversación
interact_with_chatbot()
("system", '''You are a Spanish expert chatbot of vih, who offers information resources, outreach resources and emotional support resources to users that need you to help them.
         You are helpful, inclusive, supportive, nice, educated, polite and LGTIB+ friendly. You'll always speak in Spanish because you are helping a spanish federation.
         If users ask you or try to get information which is not related to vih, you'll ask them that you can't help them because you are only specialized in vih issues. 
         Everytime you refer to vih, you'll have to use "vih" and not "VIH" (very careful with this).
         You must categorize the answer of the user in one of these three categories: "Divulgación", "Apoyo Emocional", "Recursos". Answer them by saying which category seems closer to their answer.
         ''')

### Pruebas Endpoint

In [16]:
from model import vih_chat_usuario, vih_chat_profesional

In [2]:
answer = vih_chat_usuario("Tengo vih y no sé cómo decírselo a mis padres", "Albacete","Castilla la Mancha","Amigos","Sí, y sigo tratamiento","Entre 1 y 3 meses",
                 "Hace menos de un mes", 35,"España","Cisgénero","Heterosexual","Soltería","No")
print(answer)

¡Hola! Entiendo que estás pasando por una situación difícil y quieres buscar apoyo para afrontar la revelación de tu diagnóstico de vih a tus padres. Estoy aquí para ayudarte y guiarte en este proceso.

Compartir tu diagnóstico de vih con tus seres queridos puede ser una decisión muy personal y compleja. Es normal sentir preocupación o incertidumbre sobre cómo reaccionarán tus padres. Lo importante es que te sientas preparado y elijas el momento adecuado para tener esta conversación.

Aquí te dejo algunos consejos que pueden ser útiles:

1. **Prepara el terreno**: Antes de hablar con tus padres, infórmate bien sobre el vih y el tratamiento que estás siguiendo. Esto te ayudará a sentirte más seguro y a responder a cualquier pregunta que puedan tener. Puedes visitar la página de FELGTBI+ para obtener información fiable y actualizada: [Información sobre el VIH](https://www.felgtbi.org/vih-sida/).

2. **Elige el momento adecuado**: Busca un momento en el que puedas hablar con ellos tranqui

In [17]:
pregunta = "El paciente quiere más información sobre qué tipo de virus es el vih"
answer = vih_chat_profesional(pregunta, "Sevilla","Andalucía","Por amigos","No",None,None,
                              "Hospitales y centros de salud", "Geriatría",False)
print(answer)

UndefinedColumn: column "id_sesion" of relation "interacciones" does not exist
LINE 2:     INSERT INTO interacciones (id_sesion,usuario,contenido)
                                       ^


## Pruebas few-shot prompting sin selector

In [11]:
examples_2 = [
    {
        "input": "Estoy preocupado porque creo que puedo haber contraído vih, quiero hacerme una prueba rápida",
        "output": """
                        So the final answer is: Es recomendable que acudas a un centro en el que puedas realizarte una prueba rápida, con el fin de obtener
                        un diagnóstico inicial y así ver si debes tomar algún medicamento PEP. Los centros que ofrecen dicho servicio se pueden buscar en 
                        https://felgtbi.org/que-hacemos/apoyo/salud-y-prevencion/pruebarapida/
                """,
    },
    {
        "input": "Estoy preocupado",
        "output": """
                        Are follow-up questions needed here: Yes.
                        Follow up: ¿Cuál es tu situación?
                        Intermediate answer: No sé si debería estar preocupada porque tengo VIH y estoy embarazada
                        So the final answer is: Hay tratamientos que reducen eficazmente el riesgo de transmisión del virus a su futuro hijo o hija durante
                        el embarazo y parto, y además has de tener en cuenta que no debes dar el pecho a tu bebé. 
                """,
    },
]


In [ ]:
### Ejemplo chat
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain_cohere import ChatCohere


cohere_api_key = os.getenv("COHERE_TRIAL_API_KEY")


examples = [
    {"pregunta": "¿Qué tiempo hace hoy?", "respuesta": "Hace sol y está cálido."},
    {"pregunta": "¿Cuál es la capital de Francia?", "respuesta": "La capital de Francia es París."},
    {"pregunta": "¿Cómo se dice hola en japonés?", "respuesta": "Se dice 'Konnichiwa', pringao."}
]

example_prompt = PromptTemplate(
    input_variables=["pregunta", "respuesta"],
    template="Input: {input}\nOutput: {output}\n"
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Contesta las siguientes preguntas basándote en los ejemplos.Si una pregunta coincide exactamente con un ejemplo, devuelve exactamente la misma respuesta, salvo que incluya lenguaje malsonante (insultos, palabrotas, etc.). Usa un lenguaje educado, que sea inclusivo con la comunidad LGTBI. Si no, intenta responder basándote en los ejemplos:",
    suffix="Pregunta: {pregunta_usuario}\nRespuesta:",
    input_variables=["pregunta_usuario"]
)

# Configura el modelo
chat_model = ChatCohere(cohere_api_key=cohere_api_key, temperature=0)

# Input del usuario
user_input = "¿Qué tiempo hace hoy?"

# Genera el prompt
final_prompt = few_shot_prompt.format(pregunta_usuario=user_input)

# Genera la respuesta
respuesta = chat_model.invoke(final_prompt)
print(respuesta.content)

KeyError: 'output'

In [ ]:
### Ejemplo chat
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain_cohere import ChatCohere


cohere_api_key = os.getenv("COHERE_TRIAL_API_KEY")


examples = [
    {"pregunta": "¿Qué tiempo hace hoy?", "respuesta": "Hace sol y está cálido."},
    {"pregunta": "¿Cuál es la capital de Francia?", "respuesta": "La capital de Francia es París."},
    {"pregunta": "¿Cómo se dice hola en japonés?", "respuesta": "Se dice 'Konnichiwa', pringao."}
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="input: {input}\noutput: {output}\n"
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples_2,
    example_prompt=example_prompt,
    prefix='''You are a Spanish expert chatbot of vih, who offers information resources, outreach resources and emotional support resources to users that need you to help them.
         You are helpful, inclusive, supportive, nice, educated, polite and LGTBi+ friendly. You'll always speak in Spanish because you are helping a spanish federation.
         If users ask you or try to get information which is not related to vih, you'll answer them that you can't help them because you are only specialized in vih issues.
         Everytime you refer to vih, you'll have to use "vih" and not "VIH" (very careful with this). At the bottom of the answer you must include some references from FELGTBI+ or public sources.'''
         ,
    suffix="input: {pregunta_usuario}\noutput:{output}",
    input_variables=["pregunta_usuario"]
)

# Configura el modelo
chat_model = ChatCohere(cohere_api_key=cohere_api_key, temperature=0)

# Input del usuario
user_input = "Estoy preocupado porque creo que puedo haber contraído vih, quiero hacerme una prueba rápida"

# Genera el prompt
final_prompt = few_shot_prompt.format(pregunta_usuario=user_input)

# Genera la respuesta
respuesta = chat_model.invoke(final_prompt)
print(respuesta.content)